# Intro

Terrorism is a subject largely covered in the media, and, unfortunately, we became accustomed to its presence worldwide, particularly over the last decade. Nevertheless, the problem we are facing today is not new. The source of certain conflicts dates from multiple decades, some of which are still lasting today. Our goal is to track and vizualize terrorism evolution through the past 50 years based on "The Global Terrorism Database". There are many questions we can ask ourselves about terrorism, such as "Is EU less safe nowadays ?", "Did attack mediums & reasons change over the years ?" or "Can we discriminate current/future conflictual zones ?". It would be presumptuous from us to say that we are going to solve major issues, or even predict futur attacks. However, through the exploration of the dataset, and by trying to answer those interrogations, we aim to grasp an overview and a better understanding to the evolution of terrorism.

# 1 Data understanding

First of all, we need to take a deep look into the details of our dataset to sort out the relevant data we will be using to conduct our observations. The Global Terrorism Dataset contains 135 features and approximately 170'000 entries. In order to select the label we will keep, we used the official [documentation](http://start.umd.edu/gtd/downloads/Codebook.pdf) from the dataset which describes each features precisely. Let's make a quick summary of the labels from the dataset and the one we decided to use for our project.

* `eventid` : this is the id of any entry, written as 12 numbers (first 8 are the date of event and last 4 are a sequential case number for the given day). This will be used as our index too.
* `iyear` : Year of the event
* `imonth` : Month of the event
* `iday` : Day of the event
* `country` and `country_txt` : Name of the country where the event took place
* `region` and `region_txt` : Region where the event took place
* `latitude` and `longitude` : Lattitude and Longitude values where the event took place
* `doubtterr` : boolean value set as 1 if there is a doubt to whether the incident is an act of terrorism and 0 if there is none
* `success` : boolean value set as 1 if the incident was successful or 0 if it was not. As stated in the documentation, "Success of a terrorist strike is defined according to the tangible effects of the attack. Success is not judged in terms of the larger goals of the perpetrators. For example, a bomb that exploded in a building would be counted as a success even if it did not succeed in bringing the building down or inducing government repression." 
* `suicide` : boolean value set as 1 if the attack perpetrator did not intend to escape from the attack alive, 0 otherwise.
* `attacktype1` and `attacktype1_txt` : This field captures the general method of attack and often reflects the broad class of tactics used. It consists of nine categories, which are defined below :
    1. Assassination
    2. Armed Assault
    3. Bombing/Explosion
    4. Hijacking 
    5. Hostage taking (barricade incident) 
    6. Hostage taking (kidnapping)
    7. Facility/Infrastructure Attack
    8. Unarmed Assault
    9. Unknown 
* `targtype1` and `targtype1_txt` : The target/victim type field captures the general type of target/victim. When a victim is attacked specifically because of his or her relationship to a particular person, such as a prominent figure, the target type reflects that motive. For example, if a family member of a government official is attacked because of his or her relationship to that individual, the type of target is “government.” This variable consists of the following 22 categories: <br>
    list of 22 categories
* `gname` : This field contains the name of the group that carried out the attack. In order to ensure consistency in the usage of group names for the database, the GTD database uses a standardized list of group names that have been established by project staff to serve as a reference for all subsequent entries.  
* `gname2` : This field is used to record the name of the second perpetrator when responsibility for the attack is attributed to more than one perpetrator. Conventions follow “Perpetrator Group” field.  
* `gname3` : same as for gname2
* `nperps` : This field indicates the total number of terrorists participating in the incident. (In the instance of multiple perpetrator groups participating in one case, the total number of perpetrators, across groups, is recorded). There are often discrepancies in information on this value.   
* `weaptype1,` and `weaptype1_txt` : This field records the general type of weapon used in the incident. It consists of the following categories: <br>
    llist of all weapon type
* `nkill` : This field stores the number of total confirmed fatalities for the incident. The number includes all victims and attackers who died as a direct result of the incident.   
* `nkillter`: This field stores the number of confirmed terrorists fatalities.
* `nwound` : This field records the number of confirmed non-fatal injuries to both perpetrators and victims. 
* `nwoundte` : This field records the number of confirmed non-fatal terrorists injuries. 


In [ ]:
import pandas as pd
import os
import numpy as np
import datetime

%matplotlib inline

In [ ]:
data_path = 'data'
gtd_path = os.path.join(data_path, 'globalterrorismdb_0617dist.csv')

In [ ]:
fields = ['eventid', 'iyear', 'imonth', 'iday', 'country', 'country_txt', 'region', 'region_txt', 
          'latitude', 'longitude', 'doubtterr', 'attacktype1', 'attacktype1_txt',  'success', 
          'suicide', 'weaptype1', 'weaptype1_txt', 'targtype1', 'targtype1_txt', 'gname', 'gname2', 
          'gname3', 'nperps', 'nkill', 'nkillter', 'nwound', 'nwoundte']
date_fileds = ['iyear', 'imonth', 'iday']

df = pd.read_csv(gtd_path, encoding='latin', usecols=fields, index_col='eventid', low_memory=False)

In [ ]:
print('Is index unique: {}'.format(df.index.is_unique))

In [ ]:
df.dtypes

In [ ]:
# According to documentation both month and day can be 0 (if unknown), we set them to 0
def parse_date(row):
    return datetime.date(row.iyear, int(row.imonth) if not np.isnan(row.imonth) else 1, 
                         int(row.iday) if not np.isnan(row.iday) else 1)

In [ ]:
# Count number entries with uncertain date (either month or day)
df[date_fileds] = df[date_fileds].replace(0, np.nan)
n_uncertain = np.sum(np.sum(df[date_fileds].isnull(), axis=1) != 0 )
df['date'] =df.apply(lambda x: parse_date(x), axis=1)
print('Uncertain dates: {:.2f}%, ({}/{})'.format(100*n_uncertain/len(df), n_uncertain, len(df)))

In [ ]:
print('Are country codes positive: {}'.format( all(df.country > 0)))
print('Are region codes positive: {}'.format( all(df.region > 0)))

In [ ]:
n_geo = len(df)
df.dropna(subset=('latitude', 'longitude'), inplace=True)
print('Entries without geographic coordinates droped: {:.2f}%, ({}/{})'.format(
    100*(n_geo-len(df))/n_geo, n_geo-len(df), n_geo))

In [ ]:
print('Repartition of data in %:\n{}'.format(100*df.doubtterr.value_counts()/len(df)))
df.loc[df.doubtterr < 0, 'doubtterr'] = 0
df.doubtterr = df.doubtterr.astype('category')
df.doubtterr.cat.categories = ['N_DOUBT', 'DOUBT']
print('\nRepartition of data in %(after cleaning):\n{}'.format(100*df.doubtterr.value_counts()/len(df)))

In [ ]:
print('Unique values in field: {}'.format(np.unique(df.success)))
print('Percentage of sucessful atacks: {:.2f}%'.format(100*df.success.mean()))

In [ ]:
print('Unique values in field: {}'.format(np.unique(df.suicide)))
print('Percentage of suicide atacks: {:.2f}%'.format(100*df.suicide.mean()))

In [ ]:
print('Type of attack and repartition in dataset in %')
100*df.attacktype1_txt.value_counts()/len(df)

In [ ]:
print('Type of target and repartition in dataset in %')
100*df.targtype1_txt.value_counts()/len(df)

In [ ]:
n_group = len(pd.unique(df[['gname', 'gname2', 'gname3']].values.ravel('K')))
print('Number of unique group name: {}'.format(n_group))

In [ ]:
df.loc[df.nperps < 0, 'nperps'] = np.nan
print('Percentage of entries with unknown # Perpretors {:.2f}%'.format(100*np.sum(df.nperps.isnull())/len(df)))
print('Range of # Perpretors: {} upto {}'.format(int(df.nperps.min()), int(df.nperps.max())))

In [ ]:
print('Repartition of weapon type in dataset in %')
100*df.weaptype1_txt.value_counts()/len(df)

In [ ]:
df['nkillnter'] = df.nkill-df.nkillter.fillna(0)
df.loc[df.nkillnter < 0, 'nkillnter'] = 0

print('Range total # of victims: [{}, {}]'.format(df.nkill.min(), df.nkill.max()))
print('Range # of non terrorists victims: [{}, {}]'.format(df.nkillnter.min(), df.nkillnter.max()))
print('Range # of terrorists victims: [{}, {}]'.format(df.nkillter.min(), df.nkillter.max()))

In [ ]:
df['nwoundnter'] = df.nwound-df.nwoundte.fillna(0)
df.loc[df.nwoundnter < 0, 'nwoundnter'] = 0

print('Range total # of wounded: [{}, {}]'.format(df.nwound.min(), df.nwound.max()))
print('Range # of non terrorists wounded: [{}, {}]'.format(df.nwoundnter.min(), df.nwoundnter.max()))
print('Range # of terrorists wounded: [{}, {}]'.format(df.nwoundte.min(), df.nwoundte.max()))

---

# 2 Data Visualization

## 2.1 Attacks frequency around the world over the years


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

%pylab inline
sns.set_context("notebook")

In [ ]:
year_span = 1+df.iyear.max()-df.iyear.min()
plt.figure(figsize=(16,5))
df.date.hist(bins=year_span, label='# Attacks')
plt.xlabel('Time'); plt.ylabel('# Attack'); plt.legend()

In [ ]:
np.sum(df.iyear == 1993)

According to [Codebook](http://start.umd.edu/gtd/downloads/Codebook.pdf) lack of information is the reason of lack in data for year 1993.

> In addition, users familiar with the GTD’s Data Collection Methodology are aware that incidents 
of terrorism from 1993 are not present in the GTD because they were lost prior to START’s 
compilation of the GTD from multiple data collection efforts.

In [ ]:
def get_2d_comp(df, x_col, y_col, normalize=False):
    df_month_year = df.groupby([x_col, y_col]).size().reset_index(name='frequ')
    df_month_year =  df_month_year.pivot(index=y_col, columns=x_col, values='frequ')
    if normalize:
        df_month_year = df_month_year.div(df_month_year.sum(axis=0), axis=1)
    return df_month_year

In [ ]:
df_month_year = get_2d_comp(df,'iyear', 'imonth')
df_month_year.index = ['January', 'February',  'March', 'April', 'May', 'June', 'July', 
                       'August', 'September', 'October', 'November', 'December']
plt.figure(figsize=(16,4))
sns.heatmap(df_month_year)

In [ ]:
df_reg_year = get_2d_comp(df,'iyear', 'region_txt', normalize=True)
plt.figure(figsize=(14,4))
sns.heatmap(df_reg_year.fillna(0))

In [ ]:
import folium
from folium.plugins import HeatMap, HeatMapWithTime

def get_data_longlat(df):
    data_year = df[['latitude', 'longitude']].values
    return np.concatenate((data_year, np.ones((len(data_year), 1))), axis=1)

def get_heatmap_time(df, coord=[30., 5.], zoom=2):
    data_all = []
    year_label = []
    for year, d in df.groupby('iyear'):
        data_all.append(get_data_longlat(d).tolist())
        year_label.append('Year: {}'.format(year))
    m = folium.Map(coord, tiles='stamentoner', zoom_start=zoom)
    HeatMapWithTime(data_all, index=year_label, radius=10, max_opacity=1).add_to(m)
    return m
    
def get_heatmap(df, coord=[30., 5.], zoom=2, min_opacity=0.5, blur=5):
    data = get_data_longlat(df) 
    m = folium.Map(coord, tiles='stamentoner', zoom_start=zoom)
    HeatMap(data.tolist(), radius=5, min_opacity=min_opacity, blur=blur).add_to(m)
    return m
    
rand_seed = 0
np.random.seed(rand_seed)

In [ ]:
n = 50000
id_sub = np.random.permutation(len(df))[:n]
df_sub = df.iloc[id_sub]

In [ ]:
m = get_heatmap(df_sub, coord=[30., 5.])
m

In [ ]:
m = get_heatmap_time(df_sub, coord=[30., 5.])
m

# 3. Events that marked the world

In [ ]:
thresh = 0.8*df_reg_year.max(axis=1)
thresh[thresh < 0.20] = 0.2
thresh = df_reg_year.subtract(thresh, axis=0) >= 0

plt.figure(figsize=(14,4))
sns.heatmap(thresh)

## 3.1 North America Bombings (1970)
http://time.com/4501670/bombings-of-america-burrough/

In [ ]:
df_NA_1970 = df.loc[df.region_txt=='North America']
df_NA_1970 = df_NA_1970.loc[np.logical_and(df_NA_1970.iyear >= 1970, df_NA_1970.iyear <= 1970)]

m = get_heatmap(df_NA_1970, coord=[35., -95.], zoom=4, min_opacity=0.9, blur=5)
m

## 3.2 Nothern Irland Religion conflict (1972-1973)

In [ ]:
df_EU_1972 = df.loc[df.region_txt=='Western Europe']
df_EU_1972 = df_EU_1972.loc[np.logical_and(df_EU_1972.iyear >= 1972, df_EU_1972.iyear <= 1973)]

m = get_heatmap(df_EU_1972, coord=[50., -5.], zoom=5, min_opacity=0.5, blur=1)
m

## 3.3 Nothern Irland and Basque Country (1975-1977)

In [ ]:
df_EU_1975 = df.loc[df.region_txt=='Western Europe']
df_EU_1975 = df_EU_1975.loc[np.logical_and(df_EU_1975.iyear >= 1975, df_EU_1975.iyear <= 1977)]

m = get_heatmap(df_EU_1975, coord=[50., -5.], zoom=5, min_opacity=0.5, blur=1)
m

## 3.4 Salvadoran Civil War (1981-1983)

In [ ]:
df_CA_1981 = df.loc[df.region_txt=='Central America & Caribbean']
df_CA_1981 = df_CA_1981.loc[np.logical_and(df_CA_1981.iyear >= 1981, df_CA_1981.iyear <= 1983)]

m = get_heatmap(df_CA_1981, coord=[15., -85.], zoom=5)
m

## 3.5 South America Conflicts (1984-1987)

1. Comlumbia (Farc, ELN) (https://en.wikipedia.org/wiki/Terrorism_in_Colombia)
1. Internal conflict in Peru (https://en.wikipedia.org/wiki/Internal_conflict_in_Peru)

In [ ]:
df_SA_1984 = df.loc[df.region_txt=='South America']
df_SA_1984 = df_SA_1984.loc[np.logical_and(df_SA_1984.iyear >= 1984, df_SA_1984.iyear <= 1987)]

m = get_heatmap(df_SA_1984, coord=[-20., -60.], zoom=3, min_opacity=0.3, blur=1)
m

## 3.6 Middle East (2003-2007)

1. Algeria 
2. Isreal and palestine
3. Iraq

In [ ]:
df_ME_2003 = df.loc[df.region_txt=='Middle East & North Africa']
df_ME_2003 = df_ME_2003.loc[np.logical_and(df_ME_2003.iyear >= 2003, df_ME_2003.iyear <= 2007)]

m = get_heatmap(df_ME_2003, coord=[32., 25.], zoom=4, min_opacity=0.5, blur=2)
m

## 3.7 South Asia (2008-2013)

1. Northeast India (https://en.wikipedia.org/wiki/Insurgency_in_Northeast_India)
2. Afganistan/Pakistan - Alqaida ... 
3. South east india ?

In [ ]:
df_SAsia_2008 = df.loc[df.region_txt=='South Asia']
df_SAsia_2008 = df_SAsia_2008.loc[np.logical_and(df_SAsia_2008.iyear >= 2008, df_SAsia_2008.iyear <= 2013)]

m = get_heatmap(df_SAsia_2008, coord=[20., 80.], zoom=4, min_opacity=0.3, blur=1)
m

## 3.8 Middle East (2013-Today)

1. Lybia
2. Egypt
3. Isreal palentine
4. ISIS (Syria and Iraq)

In [ ]:
df_ME_2013 = df.loc[df.region_txt=='Middle East & North Africa']
df_ME_2013 = df_ME_2013.loc[np.logical_and(df_ME_2013.iyear >= 2013, df_ME_2013.iyear <= 2017)]

m = get_heatmap(df_ME_2013, coord=[32., 25.], zoom=4, min_opacity=0.5, blur=2)
m